Repeat measure experimental designs (e.g. time series) are a valid and powerful method to control for inter-individual variation. However, conventional dimensionality reduction methods can not account for the high-correlation of each subject to itself at a later time point. This inherent correlation structure can cause subject grouping to confound or even outweigh important phenotype groupings. In the CTF tutorials we covered how tensor factorization can help. However, in many cases the time points are irregularly sampled or we may want to project new data into a previously run factorization. For these cases and more we introduced [TEMPTED](https://www.biorxiv.org/content/10.1101/2023.07.26.550749v1). There is an R implementation here but in this tutorial we will cover how to run an analysis with TEMPTED in QIIME2 / python. 

First we will download the processed data originally from [here](https://qiita.ucsd.edu/study/description/10249#). This data can be downloaded with the following links:

* **Table** (table.qza) | [download](https://github.com/biocore/gemelli/tree/master/ipynb/tutorials/ECAM-Qiita-10249/table.qza)
* **Sample Metadata** (metadata.tsv) | [download](https://github.com/biocore/gemelli/tree/master/ipynb/tutorials/ECAM-Qiita-10249/metadata.tsv)
* **Feature Metadata** (taxonomy.qza) | [download](https://github.com/biocore/gemelli/tree/master/ipynb/tutorials/ECAM-Qiita-10249/taxonomy.qza)
* **Tree** (sepp-insertion-tree.qza) | [download](https://github.com/biocore/gemelli/tree/master/ipynb/tutorials/ECAM-Qiita-10249/insertion-tree.qza)

**Note**: This tutorial assumes you have installed [QIIME2](https://qiime2.org/) using one of the procedures in the [install documents](https://docs.qiime2.org/2020.2/install/). This tutorial also assumed you have installed, [Qurro](https://github.com/biocore/qurro) and [gemelli](https://github.com/biocore/gemelli).

First, we will make a tutorial directory and download the data above and move the files to the `ECAM-Qiita-10249` directory:

```bash
mkdir ECAM-Qiita-10249
```

First we will import our data with the QIIME2 Python API. 



To run TEMPTED we only need to run one command. The required input requirements are:

1. table
    - The table is of type `FeatureTable[Frequency]` which is a table where the rows are features (e.g. ASVs/microbes), the columns are samples, and the entries are the number of sequences for each sample-feature pair.
2. sample-metadata
    - This is a QIIME2 formatted [metadata](https://docs.qiime2.org/2020.2/tutorials/metadata/) (e.g. tsv format) where the rows are samples matched to the (1) table and the columns are different sample data (e.g. time point).  
3. individual-id-column
    - This is the name of the column in the (2) metadata that indicates the individual subject/site (e.g. subject ID) that was sampled repeatedly.
4. state-column
    - This is the name of the column in the (2) metadata that indicates the numeric repeated measure (e.g., Time in months/days) or non-numeric category (i.e. decade/body-site). 
5. output-dir
    - The desired location of the output. We will cover each output independently below.  

There are also _optional_ input parameters:

*  n-components 
    * The underlying low-rank structure. The input can be
      an integer (suggested: 1 < rank < 10) [minimum 2].
      Note: as the rank increases the runtime will
      increase dramatically. [default: 3]
* replicate-handling
     * Choose how replicate samples are handled. If
       replicates aredetected, "error" causes method to
       fail; "drop" will discard all replicated samples;
       "random" chooses one representative at random from
       among replicates. [default: 'random']
* svd-centralized (bool/flag)
    * Removes the mean structure of the temporal tensor. [default: True]
* n-components-centralize 
    * Rank of approximation for average matrix in svd-centralize. [default: 1]
* smooth
    * Smoothing parameter for RKHS norm. Larger means
      smoother temporal loading functions. [default: 1e-06]
* resolution
    * Number of time points to evaluate the value of the
      temporal loading function.[default: 101]
* max-iterations 
    * Maximum number of iteration in for rank-1 calculation. [default: 20]
* epsilon
    * Convergence criteria for difference between iterations for each rank-1 calculation. [default: 0.0001]
                                                             
In this tutorial our individual-id-column is `host_subject_id` and our state-column is different time points denoted as `month` in the sample metadata. Now we are ready to run TEMPTED:

In [10]:
!qiime gemelli clr-transformation \
    --i-table ECAM-Qiita-10249/table.qza \
    --o-clr-table ECAM-Qiita-10249/clr-table.qza

!qiime gemelli tempted-factorize \
    --i-table ECAM-Qiita-10249/clr-table.qza \
    --m-sample-metadata-file ECAM-Qiita-10249/metadata.tsv \
    --p-individual-id-column 'host_subject_id' \
    --p-state-column 'month' \
    --o-individual-biplot ECAM-Qiita-10249/individual_biplot.qza \
    --o-state-loadings ECAM-Qiita-10249/state_loadings.qza \
    --o-distance-matrix ECAM-Qiita-10249/distance_matrix.qza \
    --o-svd-center ECAM-Qiita-10249/svd_center.qza

    

Saved FeatureTable[Composition] to: ECAM-Qiita-10249/clr-table.qza
Saved PCoAResults % Properties('biplot') to: ECAM-Qiita-10249/individual_biplot.qza
Saved SampleData[SampleTrajectory] to: ECAM-Qiita-10249/state_loadings.qza
Saved DistanceMatrix to: ECAM-Qiita-10249/distance_matrix.qza
Saved SampleData[SampleTrajectory] to: ECAM-Qiita-10249/svd_center.qza


We will now cover the output files:

* individual_biplot
* state_loadings
* distance_matrix
* svd_center

First, we will explore the `individual_biplot` which is an ordination where dots represent _subjects_ not _samples_ and arrows represent features (e.g. ASVs). First, we will need to aggregate the metadata by subject (i.e. collapsing the metadata of all samples from a given subject). This can be done by hand or using DataFrames in python (with pandas) or R like so:



In [8]:
import pandas as pd
from qiime2 import Metadata

# first we import the metdata into pandas
mf = pd.read_csv('ECAM-Qiita-10249/metadata.tsv', sep='\t',index_col=0)
# next we aggregate by subjects (i.e. 'host_subject_id') 
# and keep the first instance of 'diagnosis_full' by subject.
mf = mf.groupby('host_subject_id').agg({'delivery':'first','antiexposedall':'first'})
# now we save the metadata in QIIME2 format.
mf.index.name = '#SampleID'
mf.index = mf.index.astype(str)
mf.to_csv('ECAM-Qiita-10249/subject-metadata.tsv', sep='\t')
mf.head(5)


,delivery,antiexposedall
#SampleID,,
1.0,Vaginal,y
2.0,Cesarean,n
4.0,Cesarean,n
5.0,Cesarean,n
7.0,Cesarean,n


Now with the collapsed `subject-metadata.tsv` table we are ready to plot with emperor: 

In [11]:
!qiime emperor biplot\
    --i-biplot ECAM-Qiita-10249/individual_biplot.qza \
    --m-sample-metadata-file ECAM-Qiita-10249/subject-metadata.tsv \
    --m-feature-metadata-file ECAM-Qiita-10249/taxonomy.qza \
    --p-number-of-features 10\
    --o-visualization ECAM-Qiita-10249/subject_biplot.qzv


Saved Visualization to: ECAM-Qiita-10249/subject_biplot.qzv


From this visualization we can see that the birth mode clearly separate the subjects across time.

![image.png](etc/tempted_img_1.png)

We can also see that the birthmode grouping is separated entirely along the first PC (axis 2). We can now use [Qurro](https://github.com/biocore/qurro) to explore the feature loading partitions (arrows) in this biplot as a log-ratio of the original table counts. This allows us to relate these low-dimensional representations back to our original data. Additionally, log-ratios provide a nice set of data points for additional analysis such as LME models. 


In [12]:
!qiime qurro loading-plot \
    --i-table ECAM-Qiita-10249/table.qza \
    --i-ranks ECAM-Qiita-10249/individual_biplot.qza \
    --m-sample-metadata-file ECAM-Qiita-10249/metadata.tsv \
    --m-feature-metadata-file ECAM-Qiita-10249/taxonomy.qza \
    --o-visualization ECAM-Qiita-10249/qurro.qzv


Saved Visualization to: ECAM-Qiita-10249/qurro.qzv


From the Qurro output `qurro.qzv` we will simply choose the PC2 loadings above and below zero as the numerator (red ranks) and denominator (blue ranks) to create a log-ratio that differentiates the samples by birth mode status. Log-ratios can also be chosen by taxonomy or sequence identifiers (see the Qurro tutorials [here](https://github.com/biocore/qurro#tutorials) for more information). We can plot this log-ratio in Qurro with the x-axis as time and the color as birth mode, which clearly shows nice separation between phenotypes. 

![image.png](etc/tempted_img_2.png)

We can further explore these phenotype differences by exporting the `sample_plot_data.tsv` from Qurro (marked in a orange box above) which will provide the selected log-ratio values for each sample. We can then merge this `sample_plot_data` with our sample metadata in python or R. 

**Note:** Qurro will have an option to export all of the metadata or only the log-ratio data soon.


In [11]:
import pandas as pd

# import log-ratio data
metadata_one = pd.read_csv('ECAM-Qiita-10249/metadata.tsv',
                           sep='\t', index_col=0)
# import rest of the metadata
metadata_two = pd.read_csv('ECAM-Qiita-10249/sample_plot_data.tsv',
                           sep='\t', index_col=0)[['Current_Log_Ratio']]
# merge the data
log_ratio_metdata = pd.concat([metadata_two, metadata_one], axis=1)
# ensure no duplicate columns
log_ratio_metdata = log_ratio_metdata.dropna(subset=['Current_Log_Ratio'])
log_ratio_metdata.index = log_ratio_metdata.index.astype(str)
# export in QIIME2 format
log_ratio_metdata = log_ratio_metdata[['month','host_subject_id',
                                       'delivery','Current_Log_Ratio']]

log_ratio_metdata.index.name = '#SampleID'
log_ratio_metdata.to_csv('ECAM-Qiita-10249/merged_sample_plot_data.tsv', sep='\t')
log_ratio_metdata.head(2)


,month,host_subject_id,delivery,Current_Log_Ratio
#SampleID,,,,
10249.C018.14SS,16.0,18.0,Vaginal,1.604457
10249.C020.22SS,20.0,20.0,Cesarean,3.479770


As you can see above the metadata now has the added column of `Current_Log_Ratio` from Qurro. So now we will continue to explore this log-ratio by first plotting it explicitly over time with q2-longitudinal.


In [13]:
!qiime longitudinal volatility \
    --m-metadata-file ECAM-Qiita-10249/merged_sample_plot_data.tsv\
    --p-state-column month \
    --p-individual-id-column host_subject_id \
    --p-default-group-column delivery \
    --p-default-metric Current_Log_Ratio \
    --o-visualization ECAM-Qiita-10249/log_ratio_plot.qzv


Saved Visualization to: ECAM-Qiita-10249/log_ratio_plot.qzv


This demonstrates that we can recreate the separation by birth mode that we saw in both the `subject_biplot` & `state_subject_ordination`, allowing us to associate specific taxa (in the numerator or denominator) with a particular phenotype.

![image.png](etc/tempted_img_3.png)

We can test the statistical power of this log-ratio to differentiate samples by birth mode status using a linear mixed effects (LME) through q2-longitudinal. 


In [14]:
!qiime longitudinal linear-mixed-effects\
    --m-metadata-file ECAM-Qiita-10249/merged_sample_plot_data.tsv\
    --p-state-column month \
    --p-individual-id-column host_subject_id \
    --p-group-columns delivery \
    --p-metric Current_Log_Ratio \
    --o-visualization ECAM-Qiita-10249/lme_log_ratio.qzv


Saved Visualization to: ECAM-Qiita-10249/lme_log_ratio.qzv


From this LME model we can see that indeed the birthmode grouping is significant across time. 

![image.png](etc/tempted_img_4.png)

